In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
# encodeamos todas los features categoricos usando One Hot encoding


In [44]:
def normaliza_df(df, onehotencoder = None):
    print(df.shape)

    # drops -------------------------------------------------
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes (se eliminan) ----------------------------------------------
    
    # Total_Taxable_Amount = Sum rows[Total_Amount]
    
    # preparamos la columna Total_Taxable_Amount para ser aplanada
    df["Total_Amount_sum"] = df.groupby("Opportunity_ID")["Total_Amount"].transform("sum")

    df = df.drop(['Total_Taxable_Amount'], axis=1)
    df = df.drop(['Total_Amount_Currency'], axis=1)

    
    # esta columna es equivalente a oportunity_id
    df = df.drop(['Opportunity_Name'], axis=1)
    
    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)
    
    # todo: convertir estas en una ventana de tiempo
    df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], format="%m/%d/%Y")
    df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], format="%m/%d/%Y")
    
    df['Planned_Delivery_Date_diff'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
    
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)
    
    # @todo: no estoy seguro si esto aporta
    df = df.drop(['Quote_Expiry_Date'], axis=1)
    
    # @todo: convertirla a una diferencia de dias?
    df = df.drop(['Last_Modified_Date'], axis=1)
    
    # @todo: existe Opportunity_id 
    df = df.drop(['ID'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")
    
    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    #------------------------------------------
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')
    #------------------------------------------
    
    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 

    if 'Stage' in df.columns:
        onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
        onehotencoder.fit(df[categoric_cols])
    
    cat_rel = 0
    all_col_names = []
    for cat in onehotencoder.categories_:
        for col in cat:
            all_col_names.append(categoric_cols[cat_rel] + '_' + col)
        cat_rel = cat_rel + 1
    
    categorical = pd.DataFrame(onehotencoder.transform(df[categoric_cols]).toarray(), columns=all_col_names)
    
    categorical = categorical.reset_index()
    df = df.reset_index()
    
    print('ante ...')
    print(df.shape)
    print(categorical.shape)
    print(df[numeric_cols].shape)
    print(df[numeric_cols_2].shape)

    frames_to_concat = [categorical, df[numeric_cols], df[numeric_cols_2]]
    df_r =  pd.concat(frames_to_concat, axis=1)
       
    df_r = df_r.drop(columns = 'index')
    
    return df_r, onehotencoder

In [45]:
df_train = pd.read_csv('/usr/src/tp2-1/Train_TP2_Datos_2020-2C.csv')

# como vamos a hacer un merge, no queremos perder filas solo por que
# alguna posicion tiene algun dato faltante
df_train = df_train.fillna(0)

In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [47]:
df_train.loc[df_train['Opportunity_ID'] == 10]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
18,24366,Japan,None,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,Q1,2016,NaT,0,JPY,23400.0,JPY,579600.0,Closed Lost,Prod_Category_A_None
19,24367,Japan,None,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,Q1,2016,NaT,0,JPY,4400.0,JPY,579600.0,Closed Lost,Prod_Category_A_None
20,24368,Japan,None,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,Q1,2016,NaT,0,JPY,6200.0,JPY,579600.0,Closed Lost,Prod_Category_A_None
21,24369,Japan,None,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,Q1,2016,NaT,0,JPY,140000.0,JPY,579600.0,Closed Lost,Prod_Category_A_None
22,24370,Japan,None,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,Q1,2016,NaT,0,JPY,405600.0,JPY,579600.0,Closed Lost,Prod_Category_A_None


In [48]:
[ndf, onehotencoder] = normaliza_df(df_train)

(16947, 52)
ante ...
(16883, 43)
(16883, 2853)
(16883, 5)
(16883, 10)


In [49]:
#d = ndf.loc[ndf.Planned_Delivery_Date_diff == 10, :]

In [50]:
ndf.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,Territory_Australia,Territory_Austria,Territory_Belarus,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,12369,0,2018,2,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,12370,0,2016,20,0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12371,0,2016,0,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12371,0,2016,0,0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12372,0,2016,3,1


In [51]:
ndf.loc[ndf['Opportunity_ID'] == 10]

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,Territory_Australia,Territory_Austria,Territory_Belarus,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
850,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,10,0,2016,0,0
891,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,10,0,2016,0,0
892,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,10,0,2016,0,0
893,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,10,0,2016,0,0
898,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,10,0,2016,0,0


In [60]:
ndf2 = ndf.groupby('Opportunity_ID').max()
ndf2 = ndf2.reset_index()
#ndf2 = df.drop(columns = 'Opportunity_ID')
ndf2 = ndf2.fillna(0)

In [69]:

#X = ndf.dropna()

X = ndf2.copy()
y = X.pop("Stage")




# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int


In [67]:
y

0       0
1       1
2       1
3       0
4       0
       ..
9786    1
9787    0
9788    1
9789    0
9790    0
Name: Stage, Length: 9791, dtype: int64

In [70]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y)
#

In [71]:
mi_scores[0:30]

Total_Amount                             0.132032
TRF                                      0.126792
Opportunity_ID                           0.125591
Total_Amount_sum                         0.110213
Planned_Delivery_Date_diff               0.082853
ASP_(converted)                          0.077197
ASP                                      0.068472
Brand_None                               0.054309
Price_None                               0.044875
Currency_None                            0.044635
Bureaucratic_Code_0_Approval             0.042300
Product_Category_B_None                  0.042217
Size_None                                0.041894
Opportunity_Type_Opportunity_Type_19     0.039714
Account_Type_Account_Type_5              0.038718
Product_Type_None                        0.037657
Bureaucratic_Code_Bureaucratic_Code_5    0.036847
Total_Taxable_Amount_Currency_JPY        0.033073
Price_Other                              0.032980
Product_Type_Product_Type_3              0.029948


In [80]:
ndf2.to_csv('/usr/src/tp2-1/210227_tp2_train_feng.csv')

In [79]:
ndf2.shape

(9791, 2867)

In [74]:
df_test = pd.read_csv('/usr/src/tp2-1/Test_TP2_Datos_2020-2C.csv')
df_test = df_test.fillna(0)

In [75]:
[ndft, _] = normaliza_df(df_test, onehotencoder)

(2551, 51)
ante ...
(2551, 42)
(2551, 2853)
(2551, 5)
(2551, 9)


In [76]:
ndft2 = ndft.groupby('Opportunity_ID').mean()
ndft2 = ndft2.reset_index()
ndft2 = ndft2.fillna(0)

In [77]:
ndft2.shape

(1567, 2866)

In [78]:
ndft2.to_csv('/usr/src/tp2-1/210227_tp2_test_feng.csv')